<a href="https://colab.research.google.com/github/imusicmash/Stanford-Tech16-LLM-class/blob/main/Copy_of_CrewAI_Tech16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Agents with Crew AI
by Al Nevarez

### Install required pacakges


In [ ]:
%pip install --q crewai
%pip install --q -U duckduckgo-search
%pip install --q langchain_google_genai
%pip install langchain-openai

### Required Packages

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import OpenAI
from crewai import Agent, Task, Crew, Process

In [ ]:
from google.colab import userdata
# google_api_key = userdata.get('gemini_key')

### set up the LLM

In [ ]:
# # Set gemini pro as llm
# llm = ChatGoogleGenerativeAI(model="gemini-pro",
#                              verbose = True,
#                              temperature = 0.6,
#                              google_api_key=google_api_key)
llm = OpenAI(temperature=0.1, openai_api_key=userdata.get('open_ai_key'), model_name='o1-preview')

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key')

In [ ]:
llm

OpenAI(client=<openai.resources.completions.Completions object at 0x7fb142476fb0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7fb12e1c5090>, model_name='o1-preview', temperature=0.1, openai_api_key=SecretStr('**********'), openai_proxy='')

In [ ]:
llm.invoke("Hello how are you?")



NotFoundError: Error code: 404 - {'error': {'message': 'This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?', 'type': 'invalid_request_error', 'param': 'model', 'code': None}}

### setup tools

In [ ]:
# Make sure to Install duckduckgo-search for this example:
# !pip install -U duckduckgo-search

from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

In [ ]:
question_to_answer = "Conduct investment analysis on Airbnb"

**Sequential**


In [ ]:
# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal=f'Conduct comprehensive research to answer the question {question_to_answer}. With a particular focus on thinking about second order effects.',
  backstory="""You work at a leading analysis firm.
  Your expertise lies in analyzing complex issues.
  You have a knack for dissecting complex data and presenting
  actionable insights.""",
  verbose=True,
  allow_delegation=True,
  llm = llm,
  tools=[
        search_tool
      ]
)

critic = Agent(
  role='Critic',
  goal='Criticize output of research analysts to ensure the content is really insightful and not generic',
  backstory="""You are a senior analyst with an IQ of 125. You care alot about research and want to ensure insights are actionable""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[]
)

writer = Agent(
  role='Research Writer',
  goal=f'Craft a compelling research report to answer the question: {question_to_answer}',
  backstory="""You are a renowned Research Strategist, known for
  your insightful and engaging research.
  You transform complex concepts into compelling reports. Include a section on potential second order effects.""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[]
)


### Tasks to perform

In [ ]:
# Create tasks for your agents
task1 = Task(
  description=f"""Conduct a comprehensive research analysis to answer the question: {question_to_answer}""",
  agent=researcher,
  expected_output=f"Conduct a comprehensive research analysis to answer the question: {question_to_answer}"  # Add the expected output field
)

In [ ]:
# Create tasks for your agents
task2 = Task(
  description="""Critique the report and insights generated""",
  agent=critic,
  expected_output="A set of criticisms and suggested improvements"  # Add the expected output field
)

In [ ]:
task3 = Task(
  description=f"""Using the insights provided, develop a research analysis to answer the question: {question_to_answer}""",
  agent=writer,
  expected_output=f"A full research report of at least 4 paragraphs research analysis answering the question: {question_to_answer}." # Added expected output
)

### Create a Crew

In [ ]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, critic, writer],
  tasks=[task1, task2, task3],
  verbose=2, # You can set it to 1 or 2 to different logging levels,
)

In [ ]:
crew

Crew(id=5efe7803-5b95-4a39-af16-61ab1b4b0b63, process=sequential, number_of_agents=3, number_of_tasks=3)

### Kickoff the crew - let the magic happen

In [ ]:
# Get your crew to work!
result = crew.kickoff()

 [2024-08-08 23:18:35][DEBUG]: == Working Agent: Senior Research Analyst
 [2024-08-08 23:18:35][INFO]: == Starting Task: Conduct a comprehensive research analysis to answer the question: Conduct investment analysis on Airbnb


> Entering new CrewAgentExecutor chain...
As a Senior Research Analyst, it is important to approach this task with a critical and analytical mindset. I must consider all aspects and potential impacts of investing in Airbnb, including second order effects.

Action: duckduckgo_search
Action Input: {'query': 'Investment analysis on Airbnb'} 

Back To Blogs. Airbnb Investment Guide: Strategies & Opportunities. March 11, 2024. Copy link. ARTICLE SUMMARY. Unlock the secrets to successful Airbnb investing with strategic property selection, savvy data analysis, and risk mitigation to boost your profits in the competitive short-term rental market. QUICK NAVIGATION. The average cost to invest in an Airbnb rental business is between $5,000 and $10,000 for a property that yo

# Heirarchical

In [ ]:
import os
from crewai import Agent, Task, Crew, Process

# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal=f'Conduct comprehensive research to answer the question {question_to_answer}. With a particular focus on thinking about second order effects.',
  backstory="""You work at a leading analysis firm.
  Your expertise lies in analyzing complex issues.
  You have a knack for dissecting complex data and presenting
  actionable insights.""",
  verbose=True,
  allow_delegation=True,
  llm = llm,
  tools=[
        search_tool
      ]
)

critic = Agent(
  role='Critic',
  goal='Criticize output of research analysts to ensure the content is really insightful and not generic',
  backstory="""You are a senior analyst with an IQ of 125. You care alot about research and want to ensure insights are actionable""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[]
)

writer = Agent(
  role='Research Writer',
  goal=f'Craft a compelling research report to answer the question: {question_to_answer}',
  backstory="""You are a renowned Research Strategist, known for
  your insightful and engaging research.
  You transform complex concepts into compelling reports. Include a section on potential second order effects.""",
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[]
)

# Define your task
task = Task(
    description=f"Conduct a comprehensive research analysis to answer the question: {question_to_answer}.",
    expected_output="A short concise report with insights for an investment analyst",
)

# Define the manager agent
manager = Agent(
    role="Project Manager",
    goal="Efficiently manage the crew and ensure high-quality task completion",
    backstory="You're an experienced project manager, skilled in overseeing complex projects and guiding teams to success. Your role is to coordinate the efforts of the crew members, ensuring that each task is completed on time and to the highest standard.",
    allow_delegation=True,
)

# Instantiate your crew with a custom manager
crew = Crew(
    agents=[researcher, writer, critic],
    tasks=[task],
    manager_agent=manager,
    process=Process.hierarchical,
)

# Start the crew's work
result = crew.kickoff()



> Entering new CrewAgentExecutor chain...
I should use the duckduckgo_search tool to gather recent financial statements, key financial ratios, and any relevant financial news about Airbnb.

Action:
duckduckgo_search

Action Input:
{'query': 'Airbnb financial statements'}
 

The following information was filed by Airbnb, Inc. (ABNB) on Tuesday, February 13, 2024 as an 8K 2.02 statement, which is an earnings press release pertaining to results of operations and financial condition. November 01, 2023. SAN FRANCISCO, Nov. 1, 2023 /PRNewswire/ -- Airbnb, Inc. (NASDAQ: ABNB) has posted a shareholder letter containing its third quarter 2023 financial results on its Investor Relations website at https://investors.airbnb.com. Airbnb will host an audio webcast to discuss its results at 1:30 p.m. PT / 4:30 p.m. ET today. SAN FRANCISCO, Aug. 6, 2024 /PRNewswire/ -- Airbnb, Inc. (NASDAQ: ABNB) has posted a shareholder letter containing its second quarter 2024 financial results on its Investor Rel

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

**Investment Analysis Report on Airbnb**

---

**Executive Summary**

Airbnb has shown remarkable growth and resilience in the travel and hospitality industry. This report provides a comprehensive investment analysis of Airbnb, highlighting its financial performance, business model, market trends, competitive landscape, and potential risks. The insights presented here are intended to inform investment analysts and decision-makers.

---

**Financial Performance**

Recent financial data indicates strong growth for Airbnb:

- Revenue increased by 18% in the third quarter of 2023 compared to the same period in 2022.
- Net income surged by 126% in the same period, improving the profit margin to 12% from 6.4% in 2022.
- Historical data shows a steady increase in revenue, with a 19.2% year-on-year increase in 2023 and a significant net income increase of 638% in 2022, indicating strong recovery from previous losses.
- Total revenue in 2023 was $9.92 billion, up from $8.4 billion in 2022.

These metrics suggest a robust growth trajectory and underline Airbnb's potential for future performance.

---

**Business Model and Profitability Drivers**

Airbnb operates a two-sided marketplace connecting hosts with guests seeking short-term accommodations. Key revenue streams include:

- **Booking Fees**: Charged to guests, typically 6-12% of the total booking cost.
- **Host Service Fees**: Charged to hosts, typically 3-5% of the total booking cost.
- **Ancillary Services**: Includes experiences and premium services like Airbnb Plus.

Key drivers of profitability include:

- **Occupancy Rates**: Higher occupancy rates enhance revenue for both Airbnb and hosts.
- **Average Booking Values**: Higher values contribute to increased revenue.
- **Cost Management Strategies**: Effective cost management improves profitability.

---

**Market Trends**

Market trends supporting Airbnb's growth:

- **Experiential Travel**: Increasing demand for unique and authentic travel experiences, especially among millennials and Gen Z.
- **Home-sharing Popularity**: Steady increase in home-sharing preference over traditional hotels.
- **Market Projections**: The global vacation rental market is expected to reach $113.9 billion by 2027, with Airbnb holding a significant market share.

These trends position Airbnb favorably for continued growth.

---

**Competitive Landscape**

Airbnb maintains a strong competitive position:

- **Market Share**: Holds a 20% market share in the vacation rental industry.
- **Revenue Growth**: Year-over-year revenue growth of 18% outperforms competitors like Booking.com and Expedia.
- **Financial Health**: Lower debt-to-equity ratio and significant increases in net income and corporate revenue.

Airbnb's unique business model, adaptability, and brand recognition provide a competitive edge.

---

**Potential Risks**

Key risks include:

- **Regulatory Changes**: Potential new laws or regulations in key markets could increase operating costs and limit expansion.
- **Shifts in Consumer Preferences**: Post-pandemic travel trends may change, affecting demand for Airbnb's offerings.

These risks necessitate close monitoring by investors.

---

**Conclusion**

Airbnb presents a strong investment opportunity with a promising growth trajectory, robust financial performance, and a favorable market position. However, potential risks such as regulatory changes and shifts in consumer preferences should be carefully considered. Overall, Airbnb is well-positioned for continued success in the short-term rental market.

---

This comprehensive analysis provides valuable insights for decision-makers and highlights Airbnb's potential as a strong investment option.